# Bottle Cap Detection - Model Development
**Author:** Wikan Priambudi
**Goal:** Develop a lightweight detection model (<10ms inference) for Raspberry Pi 5.

## Setting wandb

In [7]:
import wandb
from ultralytics import YOLO

# Login ke WandB (Nanti akan muncul input box untuk API Key)
wandb.login()

# 3. Pastikan integrasi Ultralytics aktif
from ultralytics import settings
settings.update({'wandb': True})
print("WandB Integration: Active")

wandb: Currently logged in as: wikan (wikan-project) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


WandB Integration: Active


### ONNX

In [8]:
from ultralytics import YOLO, settings
import onnxruntime as ort
import numpy as np
import time
import wandb
import os

# --- KONFIGURASI GLOBAL ---
PROJECT_NAME = "Bottle-Cap-Detection"
DATA_YAML = '../datasets/bottle_cap/data.yaml'

# Aktifkan integrasi Ultralytics -> W&B
settings.update({'wandb': True})

def run_experiment(model_name, exp_name, img_size=320, use_safe_aug=False):
    """
    Fungsi SATU PINTU: Training -> Export ONNX -> Benchmark Speed -> Upload ke WandB.
    """
    print(f"\n🚀 START EXPERIMENT: {exp_name} | Size: {img_size} | Safe Aug: {use_safe_aug}")

    # ===========================
    # 1. INISIALISASI WANDB & SIMPAN ID (INI YANG HILANG TADI)
    # ===========================
    # Bersihkan sesi nyangkut
    if wandb.run is not None: wandb.finish()
    
    # Init run baru
    run = wandb.init(project=PROJECT_NAME, name=exp_name, job_type="train_and_eval")
    my_run_id = run.id  # <--- KITA SIMPAN KUNCINYA DISINI!
    
    # Kita biarkan sesi ini AKTIF saat masuk ke model.train
    # Ultralytics otomatis akan mendeteksi sesi aktif ini dan "menumpang" di situ.
    
    try:
        # ===========================
        # TAHAP 2: TRAINING
        # ===========================
        model = YOLO(model_name)

        train_args = {
            "data": DATA_YAML,
            "epochs": 50, # Bisa dinaikkan nanti
            "imgsz": img_size,
            "batch": 8,
            "patience": 15,
            "project": PROJECT_NAME, 
            "name": exp_name,
            "device": "cpu",
            "plots": True,
            "exist_ok": True,
        }

        if use_safe_aug:
            print("🛡️ Mengaktifkan SAFE MODE...")
            train_args.update({
                "mosaic": 0.0, "scale": 0.1, "degrees": 0.0,
                "hsv_h": 0.0, "hsv_s": 0.0, "hsv_v": 0.0
            })

        # Training
        # Ultralytics akan menggunakan run yang sudah kita init di atas
        model.train(**train_args)

        print("📊 Running Validation...")
        metrics = model.val()
        
        # ===========================
        # TAHAP 3: EXPORT ONNX
        # ===========================
        print("📦 Exporting to ONNX...")
        model.export(format='onnx', dynamic=False, opset=12, imgsz=img_size)

        onnx_path = os.path.join(PROJECT_NAME, exp_name, 'weights', 'best.onnx')
        if not os.path.exists(onnx_path):
            onnx_path = f"{PROJECT_NAME}/{exp_name}/weights/best.onnx"

        if not os.path.exists(onnx_path):
            print("❌ File ONNX tidak ditemukan, skip benchmark.")
            return 0

        # ===========================
        # TAHAP 4: BENCHMARK ONNX
        # ===========================
        print(f"⏱️ Running Benchmark on: {onnx_path}")
        session = ort.InferenceSession(onnx_path)
        input_name = session.get_inputs()[0].name
        dummy_input = np.random.randn(1, 3, img_size, img_size).astype(np.float32)

        # Warmup
        for _ in range(10): session.run(None, {input_name: dummy_input})

        # Testing loop
        start = time.time()
        for _ in range(50): session.run(None, {input_name: dummy_input})
        
        avg_ms = (time.time() - start) / 50 * 1000
        fps = 1000 / avg_ms

        print(f"✅ Speed Result: {avg_ms:.2f} ms ({fps:.1f} FPS)")

        # ===========================
        # TAHAP 5: LOGGING & FINISH
        # ===========================
        # Cek status run. Jika Ultralytics menutupnya, kita buka lagi pakai ID lama.
        if wandb.run is None:
            print(f"🔄 Re-connecting ke Run ID: {my_run_id}...")
            wandb.init(id=my_run_id, project=PROJECT_NAME, resume="must")

        # Log data benchmark
        log_data = {
            "inference/latency_ms": avg_ms,
            "inference/onnx_fps": fps,
            "model/size_mb": os.path.getsize(onnx_path) / (1024 * 1024),
            "val/mAP50": metrics.box.map50 
        }
        
        wandb.log(log_data)
        
        # Update summary agar muncul di bar chart dashboard
        for key, value in log_data.items():
            wandb.run.summary[key] = value

        print("✅ Data benchmark berhasil dikirim ke WandB!")
        return avg_ms

    except Exception as e:
        print(f"❌ Error terjadi: {e}")
        raise e
        
    finally:
        # Pastikan sesi selalu ditutup di akhir
        if wandb.run is not None:
            wandb.finish()
            print("🔒 Sesi WandB ditutup.")

### FP16

In [9]:
from ultralytics import YOLO, settings
import numpy as np
import time
import wandb
import os
import shutil
from pathlib import Path

# --- KONFIGURASI GLOBAL ---
PROJECT_NAME = "Bottle-Cap-Detection"
DATA_YAML = '../datasets/bottle_cap/data.yaml'

# Pastikan setting Ultralytics connect ke WandB
settings.update({'wandb': True})

def run_experiment_fp16(model_name, exp_name, img_size=320, use_safe_aug=False):
    """
    Fungsi SATU PINTU: Training -> Export OpenVINO FP16 -> Benchmark -> Upload WandB.
    """
    print(f"\n🚀 START FP16 EXPERIMENT: {exp_name} | Size: {img_size} | Safe Aug: {use_safe_aug}")

    # 1. INISIALISASI WANDB
    if wandb.run is not None: wandb.finish()
    
    run = wandb.init(project=PROJECT_NAME, name=exp_name, job_type="train_and_eval_fp16")
    my_run_id = run.id
    
    try:
        # ===========================
        # TAHAP 2: TRAINING
        # ===========================
        model = YOLO(model_name)
        
        train_args = {
            "data": DATA_YAML,
            "epochs": 50,  
            "imgsz": img_size,
            "batch": 8,
            "patience": 15,
            "project": PROJECT_NAME, 
            "name": exp_name,
            "device": "cpu", 
            "plots": True,
            "exist_ok": True,
        }

        if use_safe_aug:
            print("🛡️ Mengaktifkan SAFE MODE (Augmentasi Warna OFF)...")
            train_args.update({
                "mosaic": 0.0, "scale": 0.1, "degrees": 0.0,
                "hsv_h": 0.0, "hsv_s": 0.0, "hsv_v": 0.0
            })
        
        # Training
        model.train(**train_args)
        
        print("📊 Running Validation...")
        metrics = model.val()
        
        # ===========================
        # TAHAP 3: EXPORT OPENVINO FP16
        # ===========================
        print("\n📦 Exporting to OpenVINO FP16...")
        
        # Hapus folder lama jika ada (untuk menghindari error permission di Windows)
        save_dir = Path(f'{PROJECT_NAME}/{exp_name}/weights')
        ov_old_dirs = list(save_dir.glob('*openvino*'))
        for d in ov_old_dirs:
            try:
                shutil.rmtree(d)
                print("   (Folder export lama dihapus)")
            except:
                pass

        # Export command dengan half=True
        ov_path_result = model.export(
            format='openvino', 
            imgsz=img_size, 
            half=True,   # <--- INI KUNCI FP16 (Half Precision)
            dynamic=False
        )
        
        # Cari folder hasil export (biasanya best_openvino_model)
        ov_dirs = list(save_dir.glob('*openvino*'))
        if not ov_dirs:
            print("❌ Folder OpenVINO tidak ditemukan, skip benchmark.")
            return 0
            
        model_ov_path = str(ov_dirs[0]) # Path ke folder model OpenVINO

        # ===========================
        # TAHAP 4: BENCHMARK OPENVINO FP16
        # ===========================
        print(f"⏱️ Running Benchmark on: {model_ov_path}")
        
        # Load model OpenVINO menggunakan Ultralytics Wrapper
        # (Ini cara paling stabil karena otomatis handle preprocessing)
        model_ov = YOLO(model_ov_path, task='detect')
        
        # Kita butuh gambar dummy/real untuk predict() method Ultralytics
        # Opsi A: Pakai gambar random (numpy)
        dummy_img = np.random.randint(0, 255, (img_size, img_size, 3), dtype=np.uint8)
        
        # Warmup (10x)
        for _ in range(10): 
            model_ov.predict(source=dummy_img, imgsz=img_size, verbose=False)

        # Testing (50x)
        start = time.time()
        for _ in range(50):
            model_ov.predict(source=dummy_img, imgsz=img_size, verbose=False)
        
        avg_ms = (time.time() - start) / 50 * 1000
        fps = 1000 / avg_ms
        
        print(f"✅ Speed Result (FP16): {avg_ms:.2f} ms ({fps:.1f} FPS)")
        
        # ===========================
        # TAHAP 5: LOGGING & FINISH
        # ===========================
        if wandb.run is None:
            print(f"🔄 Re-connecting ke Run ID: {my_run_id}...")
            wandb.init(id=my_run_id, project=PROJECT_NAME, resume="must")
            
        # Hitung ukuran folder model (karena OpenVINO isinya folder .xml & .bin)
        model_size_mb = sum(f.stat().st_size for f in Path(model_ov_path).glob('**/*') if f.is_file()) / (1024 * 1024)

        log_data = {
            "inference/latency_ms": avg_ms,
            "inference/openvino_fp16_fps": fps, # Ganti nama biar beda dengan ONNX
            "model/size_mb": model_size_mb,
            "val/mAP50": metrics.box.map50,
            "precision": "FP16"
        }
        
        wandb.log(log_data)
        
        # Update summary
        for key, value in log_data.items():
            wandb.run.summary[key] = value
            
        print("✅ Data FP16 berhasil dikirim ke WandB!")
        return avg_ms

    except Exception as e:
        print(f"❌ Experiment Failed: {e}")
        raise e
        
    finally:
        if wandb.run is not None:
            wandb.finish()
            print("🔒 WandB Session Closed.")

## YOLOv8n

In [10]:
# Jalankan Eksperimen 1: Ukuran 320, Augmentasi Aman
run_experiment(
    model_name='yolov8n.pt', 
    exp_name='yolov8n_640', 
    img_size=640, 
    use_safe_aug=False
)


🚀 START EXPERIMENT: yolov8n_640 | Size: 640 | Safe Aug: False


Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_640, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plots=Tru

wandb: WARNING Tried to log to step 50 that is less than the current step 51. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


lr/pg0,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇██████████▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▃▄▄▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▆▅▅▄▄▄▃▂
lr/pg2,▁▂▂▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▆▅▅▄▄▃▃▂
metrics/mAP50(B),▁▁▁▁▁▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███████████
metrics/mAP50-95(B),▁▁▁▁▁▂▃▃▄▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇█████████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▆▆███████████
metrics/recall(B),▁▂▂▃▅████████████████████▇▇▇▁▁▅▅▅▆▆▆▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 1.90.6 ms, read: 2.30.5 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 2.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 1.8it/s 0.5s
                   all          3         19      0.956       0.85      0.985      0.892
            light_blue          1          5      0.868          1      0.995      0.935
             dark_blue          1          7          1      0.862      0.995      0.836
                others          1          7          1      0.689      0.964      0.903
Speed: 0.9ms preprocess, 96.8ms inference, 0.0

✅ Data benchmark berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/onnx_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,124.7909
inference/onnx_fps,8.0134
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.981
metrics/mAP50-95(B),0.88368


🔒 Sesi WandB ditutup.


124.79090213775635

### Change size into 320

In [11]:
# Jalankan Eksperimen 1: Ukuran 320, Augmentasi Aman
run_experiment(
    model_name='yolov8n.pt', 
    exp_name='yolov8n_320', 
    img_size=320, 
    use_safe_aug=False
)


🚀 START EXPERIMENT: yolov8n_320 | Size: 320 | Safe Aug: False


Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_320, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plots=Tru

lr/pg0,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▃▄▅▅▅▆▆▇▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg2,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇██████████▇▇▇▇▆▆▆▅▅▄▄▃▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▁▁▁▃▄▄▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▁▃▃▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇███████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████▇▇▅▅▆▆▆▆▆▆▅
metrics/recall(B),▅▆▇█████████████▁▁▁▁▁▁▁▁▁▁▂▂▂▂▅▅▅▇▇▇▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 1.90.8 ms, read: 2.00.5 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 8.0it/s 0.1s
                   all          3         19      0.658      0.869      0.747      0.609
            light_blue          1          5      0.396          1      0.555      0.455
             dark_blue          1          7      0.576      0.973      0.709       0.53
                others          1          7          1      0.633      0.978      0.842
Speed: 0.2ms preprocess, 25.5ms inference, 0.0ms loss,

✅ Data benchmark berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/onnx_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,24.8114
inference/onnx_fps,40.30405
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.75276
metrics/mAP50-95(B),0.61058


🔒 Sesi WandB ditutup.


24.8114013671875

### Augmentation

In [12]:
# Jalankan Eksperimen 1: Ukuran 320, Augmentasi Aman
run_experiment(
    model_name='yolov8n.pt', 
    exp_name='yolov8n_320_aug', 
    img_size=320, 
    use_safe_aug=True
)


🚀 START EXPERIMENT: yolov8n_320_aug | Size: 320 | Safe Aug: True


🛡️ Mengaktifkan SAFE MODE...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolov8n_320_aug, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

lr/pg0,▁▂▂▃▃▄▄▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▆▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇█████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg2,▁▂▃▄▄▅▅▅▆▆▇▇▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▁▂▂▃▃▄▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇██████████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▂▃▃▃▃▄▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇███████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃█▇▇▇▆▆▅▅▅▅▅▅▅▇▇▇▇▇▇▇▇
metrics/recall(B),▅▆██████████████▃▃▁▁▁▁▄▄▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 1.60.7 ms, read: 1.80.7 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 3.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 5.4it/s 0.2s
                   all          3         19      0.812      0.924      0.927      0.802
            light_blue          1          5      0.702          1      0.962      0.799
             dark_blue          1          7      0.734      0.792      0.825      0.706
                others          1          7          1       0.98      0.995      0.902
Speed: 0.2ms preprocess, 23.5ms inference, 0.0

✅ Data benchmark berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/onnx_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,11.96108
inference/onnx_fps,83.6045
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.92722
metrics/mAP50-95(B),0.80243


🔒 Sesi WandB ditutup.


11.961078643798828

### FP16

In [13]:
# 1. Jalankan FP16 dengan Size 320
run_experiment_fp16(
    model_name='yolov8n.pt',       # atau 'yolo11n.pt'
    exp_name='yolov8n_320_fp16', 
    img_size=320, 
    use_safe_aug=True
)


🚀 START FP16 EXPERIMENT: yolov8n_320_fp16 | Size: 320 | Safe Aug: True


🛡️ Mengaktifkan SAFE MODE (Augmentasi Warna OFF)...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolov8n_320_fp16, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

lr/pg0,▁▂▂▃▃▄▄▅▅▅▇▇▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg2,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇███████████▇▇▇▇▆▆▆▅▅▄▄▃▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▁▁▂▃▃▃▄▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇█████████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▂▃▃▃▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇███████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃██▇▇▆▆▅▅▅▅▅▅▅▇▇▇▇▇▇▇▇▇
metrics/recall(B),▅▆██████████████▃▃▁▁▁▁▄▄▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 1.60.5 ms, read: 2.10.7 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 7.7it/s 0.1s
                   all          3         19      0.812      0.924      0.927      0.802
            light_blue          1          5      0.702          1      0.962      0.799
             dark_blue          1          7      0.734      0.792      0.825      0.706
                others          1          7          1       0.98      0.995      0.902
Speed: 0.3ms preprocess, 28.2ms inference, 0.0ms loss,

✅ Data FP16 berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/openvino_fp16_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,10.05448
inference/openvino_fp16_fps,99.45817
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.92722
metrics/mAP50-95(B),0.80243


🔒 WandB Session Closed.


10.054478645324707

### Yolov8n 256

In [14]:
# 1. Coba Size 256 (Sweet Spot?)
run_experiment(
    model_name='yolov8n.pt',  # Pastikan ini menunjuk ke checkpoint terbaikmu
    exp_name='yolov8n_256',
    img_size=256,             # <--- Turunkan resolusi
    use_safe_aug=True
)


🚀 START EXPERIMENT: yolov8n_256 | Size: 256 | Safe Aug: True


🛡️ Mengaktifkan SAFE MODE...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolov8n_256, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15,

lr/pg0,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▆▅▅▄▄▃▃▂
lr/pg2,▁▂▃▃▄▅▅▅▆▆▇▇▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▁▁▁▁▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇███
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▁▂▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▆▆▆█
metrics/recall(B),▂▃▄▅▆▇▇▇██████████████▂▂▁▁▂▂▂▄▆▆▆▇▇▇▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 1.50.4 ms, read: 2.30.9 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 1.5Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 5.9it/s 0.2s
                   all          3         19       0.84      0.886      0.933      0.789
            light_blue          1          5          1      0.657      0.872      0.727
             dark_blue          1          7      0.577          1      0.933      0.824
                others          1          7      0.943          1      0.995      0.817
Speed: 0.3ms preprocess, 36.1ms inference, 0.0

✅ Data benchmark berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/onnx_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,35.56535
inference/onnx_fps,28.11725
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.9314
metrics/mAP50-95(B),0.78792


🔒 Sesi WandB ditutup.


35.56535243988037

## YOLOv10n

In [15]:
# Jalankan Eksperimen 1: Ukuran 320, Augmentasi Aman
run_experiment(
    model_name='yolov10n.pt', 
    exp_name='yolov10n_320', 
    img_size=320, 
    use_safe_aug=False
)


🚀 START EXPERIMENT: yolov10n_320 | Size: 320 | Safe Aug: False


Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov10n_320, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plots=T

lr/pg0,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▆▆▆▅▅▄▄▄▃▂
lr/pg1,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇██████████▇▇▇▇▆▆▆▅▅▄▄▃▃▂
lr/pg2,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▄▄▄▄▅▅▆▆▇▇▇▇▇▇████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇█
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████
metrics/recall(B),▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▅▅▆▆▆▇▇▇▇▇███▂▂▂
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
YOLOv10n summary (fused): 102 layers, 2,265,753 parameters, 0 gradients, 6.5 GFLOPs
val: Fast image access  (ping: 1.60.5 ms, read: 1.80.7 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 2.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 4.1it/s 0.2s
                   all          3         19      0.891      0.198      0.666      0.506
            light_blue          1          5          1          0      0.415      0.313
             dark_blue          1          7          1          0      0.735      0.489
                others          1          7      0.674      0.594       0.85      0.715
Speed: 0.3ms preprocess, 61.0ms inference,

✅ Data benchmark berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/onnx_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,34.95661
inference/onnx_fps,28.60689
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.66643
metrics/mAP50-95(B),0.50576


🔒 Sesi WandB ditutup.


34.9566125869751

## YOLO9t

In [16]:
# Jalankan Eksperimen 1: Ukuran 320, Augmentasi Aman
run_experiment(
    model_name='yolov9t.pt', 
    exp_name='yolov9t_320', 
    img_size=320, 
    use_safe_aug=False
)


🚀 START EXPERIMENT: yolov9t_320 | Size: 320 | Safe Aug: False


Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov9t.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov9t_320, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plots=Tru

lr/pg0,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇██████████▇▇▇▇▆▆▆▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▃▄▅▅▅▆▆▇▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg2,▁▂▂▃▃▄▄▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▆▅▅▄▄▃▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▂▂▂▃▃▄▄▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇██████████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▄▄▅▅▆▆▆▆▆▆▇▇██████████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▆▆████▇▇▇▇▇▇▇▇▆▆▆▆▆▆
metrics/recall(B),▁▂▂▂▃▆▇████████▃▃▃▄▄▄▄▂▂▂▂▂▂▂▅▅▆▆▆▆▆▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
YOLOv9t summary (fused): 197 layers, 1,971,369 parameters, 0 gradients, 7.6 GFLOPs
val: Fast image access  (ping: 2.00.8 ms, read: 1.90.4 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 1.5Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 2.8it/s 0.4s
                   all          3         19      0.723      0.848      0.965      0.807
            light_blue          1          5      0.304          1      0.995      0.868
             dark_blue          1          7      0.866      0.714      0.905      0.673
                others          1          7          1      0.831      0.995      0.882
Speed: 0.3ms preprocess, 95.8ms inference, 

✅ Data benchmark berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/onnx_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,46.06809
inference/onnx_fps,21.707
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.965
metrics/mAP50-95(B),0.80777


🔒 Sesi WandB ditutup.


46.06808662414551

### Augmentation

In [17]:
# Jalankan Eksperimen 1: Ukuran 320, Augmentasi Aman
run_experiment(
    model_name='yolov9t.pt', 
    exp_name='yolov9t_320_aug', 
    img_size=320, 
    use_safe_aug=True
)


🚀 START EXPERIMENT: yolov9t_320_aug | Size: 320 | Safe Aug: True


🛡️ Mengaktifkan SAFE MODE...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov9t.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolov9t_320_aug, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

lr/pg0,▁▂▂▃▃▄▄▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▃▄▄▅▅▅▆▇▇▇▇███████████▇▇▇▇▆▆▆▅▅▄▄▃▃▂
lr/pg2,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇███████████▇▇▇▇▆▆▆▅▅▄▄▄▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▁▁▂▂▂▂▃▄▄▄▆▆▇▇▇▇▇▇███████████████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▃▃▅▅▆▆▆▆▆▆▆▆▆▇▇██████████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▆███████▇▇▇███████████
metrics/recall(B),▁▁▂▃▄▇██████████▄▄▂▂▂▂▅▅▆▆▇▇▇▇██████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
YOLOv9t summary (fused): 197 layers, 1,971,369 parameters, 0 gradients, 7.6 GFLOPs
val: Fast image access  (ping: 2.61.7 ms, read: 2.20.6 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 3.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 2.8it/s 0.4s
                   all          3         19      0.903      0.966      0.995      0.884
            light_blue          1          5      0.853          1      0.995      0.975
             dark_blue          1          7      0.856          1      0.995      0.814
                others          1          7          1      0.898      0.995      0.862
Speed: 0.3ms preprocess, 96.4ms inference, 

✅ Data benchmark berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/onnx_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,42.73753
inference/onnx_fps,23.39864
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.995
metrics/mAP50-95(B),0.87711


🔒 Sesi WandB ditutup.


42.73752689361572

### FP16

In [18]:
# 1. Jalankan FP16 dengan Size 320
run_experiment_fp16(
    model_name='yolov9t.pt',       # atau 'yolo11n.pt'
    exp_name='yolov9t_320_fp16', 
    img_size=320, 
    use_safe_aug=True
)


🚀 START FP16 EXPERIMENT: yolov9t_320_fp16 | Size: 320 | Safe Aug: True


🛡️ Mengaktifkan SAFE MODE (Augmentasi Warna OFF)...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov9t.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolov9t_320_fp16, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

lr/pg0,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▃▄▄▅▅▆▆▇▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg2,▁▂▂▃▃▄▄▅▅▅▆▇▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▁▂▂▂▂▃▄▄▆▆▇▇▇▇▇▇█████████████████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▅▅▆▆▆▆▆▆▆▆▆▆▆▇▇██████████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆███████▇▇▇████████████
metrics/recall(B),▁▁▂▃▄▇█████████▄▄▂▂▂▂▅▅▆▆▇▇▇▇▇██████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
YOLOv9t summary (fused): 197 layers, 1,971,369 parameters, 0 gradients, 7.6 GFLOPs
val: Fast image access  (ping: 1.60.6 ms, read: 2.20.8 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 3.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 2.8it/s 0.4s
                   all          3         19      0.903      0.966      0.995      0.884
            light_blue          1          5      0.853          1      0.995      0.975
             dark_blue          1          7      0.856          1      0.995      0.814
                others          1          7          1      0.898      0.995      0.862
Speed: 0.2ms preprocess, 95.3ms inference, 

✅ Data FP16 berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/openvino_fp16_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,45.73826
inference/openvino_fp16_fps,21.86353
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.995
metrics/mAP50-95(B),0.87711


🔒 WandB Session Closed.


45.73826313018799

## YOLOv5nu

In [19]:
# Jalankan Eksperimen 1: Ukuran 320, Augmentasi Aman
run_experiment(
    model_name='yolov5nu.pt', 
    exp_name='yolo5nu_320', 
    img_size=320, 
    use_safe_aug=False
)


🚀 START EXPERIMENT: yolo5nu_320 | Size: 320 | Safe Aug: False


Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov5nu.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo5nu_320, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plots=Tr

lr/pg0,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▆▆▆▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▄▄▅▅▅▆▆▇▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg2,▁▂▂▃▃▄▄▅▅▅▆▇▇▇▇███████████▇▇▇▇▆▆▅▅▅▄▄▃▃▂
metrics/mAP50(B),▁▁▁▁▁▁▂▂▂▂▂▂▂▂▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████
metrics/mAP50-95(B),▁▁▁▁▁▁▂▂▂▂▂▂▂▂▄▅▆▆▇▇▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅██▅▅▅▅▆▆▆▆▆▆▆▆▆▆
metrics/recall(B),▄▄▄▅▅█████████████████▃▃▁▁▁▁▁▂▅▅▅▅▅▅▆▆▆▆
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
YOLOv5n summary (fused): 84 layers, 2,503,529 parameters, 0 gradients, 7.1 GFLOPs
val: Fast image access  (ping: 1.80.8 ms, read: 1.90.8 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 2.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 4.7it/s 0.2s
                   all          3         19       0.54      0.795      0.665      0.539
            light_blue          1          5      0.257          1      0.442      0.358
             dark_blue          1          7      0.364          1      0.558       0.43
                others          1          7          1      0.386      0.995      0.828
Speed: 1.4ms preprocess, 47.1ms inference, 0

✅ Data benchmark berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/onnx_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,34.7533
inference/onnx_fps,28.77424
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.66505
metrics/mAP50-95(B),0.53871


🔒 Sesi WandB ditutup.


34.75330352783203

## YOLO11n

In [20]:
# Jalankan Eksperimen 1: Ukuran 320, Augmentasi Aman
run_experiment(
    model_name='yolo11n.pt', 
    exp_name='yolo11n_320', 
    img_size=320, 
    use_safe_aug=False
)


🚀 START EXPERIMENT: yolo11n_320 | Size: 320 | Safe Aug: False


Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_320, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plots=Tru

lr/pg0,▁▂▃▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▆▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▆▆▆▅▅▄▄▃▃▂
lr/pg2,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇█████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▁▁▂▃▃▃▃▅▅▅▅▅▅▆▆▆▇▇▇████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▂▂▂▂▂▁▁▁▁▂▂▃▃▃▃▄▄▄▄▄▅▅▆▆▆▇▇████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███████
metrics/recall(B),▂▃▃▅▅▅▅▆▇▇▇██████████████████████▁▁▁▁▁▁▂
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 1.60.5 ms, read: 2.20.6 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 1.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 4.1it/s 0.2s
                   all          3         19      0.788      0.433      0.866      0.737
            light_blue          1          5      0.364          1      0.906      0.781
             dark_blue          1          7          1          0      0.698       0.59
                others          1          7          1      0.299      0.995       0.84
Speed: 0.5ms preprocess, 52.1ms inference, 

✅ Data benchmark berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/onnx_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,41.10622
inference/onnx_fps,24.32722
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.8665
metrics/mAP50-95(B),0.73702


🔒 Sesi WandB ditutup.


41.10621929168701

### Augmentation

In [21]:
# Jalankan Eksperimen 1: Ukuran 320, Augmentasi Aman
run_experiment(
    model_name='yolo11n.pt', 
    exp_name='yolo11n_320_aug', 
    img_size=320, 
    use_safe_aug=True
)


🚀 START EXPERIMENT: yolo11n_320_aug | Size: 320 | Safe Aug: True


🛡️ Mengaktifkan SAFE MODE...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolo11n_320_aug, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

lr/pg0,▁▂▂▃▃▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▃▄▄▅▅▅▆▇▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg2,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇██████████▇▇▇▇▆▆▅▅▅▄▄▃▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▄▄▆▆▆▇▇█████████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▅▅▅▇█████████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████▇▇▇▇
metrics/recall(B),▃▃▃▄▅▅▆▆▆▇▇██████████████▁▁▁▂▂▂▂▂▂▂▂▅▅▅▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 1.90.8 ms, read: 2.20.5 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 1.4Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 4.2it/s 0.2s
                   all          3         19      0.917      0.932      0.995      0.886
            light_blue          1          5       0.77          1      0.995      0.955
             dark_blue          1          7      0.981          1      0.995      0.858
                others          1          7          1      0.797      0.995      0.846
Speed: 0.3ms preprocess, 53.4ms inference, 

✅ Data benchmark berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/onnx_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,38.22703
inference/onnx_fps,26.1595
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.995
metrics/mAP50-95(B),0.88647


🔒 Sesi WandB ditutup.


38.22702884674072

### FP16

In [22]:
# 1. Jalankan FP16 dengan Size 320
run_experiment_fp16(
    model_name='yolo11n.pt',       # atau 'yolo11n.pt'
    exp_name='yolo11n_320_fp16', 
    img_size=320, 
    use_safe_aug=True
)


🚀 START FP16 EXPERIMENT: yolo11n_320_fp16 | Size: 320 | Safe Aug: True


🛡️ Mengaktifkan SAFE MODE (Augmentasi Warna OFF)...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolo11n_320_fp16, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

lr/pg0,▁▂▃▃▄▄▅▅▆▆▇▇▇▇▇██████████▇▇▇▇▇▆▆▅▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▅▅▄▄▃▃▂
lr/pg2,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▆▅▅▄▄▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▄▄▆▆▇▇█████████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▃▃▃▃▅▅▅▇▇█████████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███████████▇▇▇▇
metrics/recall(B),▃▃▃▄▅▅▆▆▆▇▇██████████████▁▁▁▂▂▂▂▂▂▂▂▅▅▅▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 1.90.8 ms, read: 1.90.3 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 1.5Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 4.6it/s 0.2s
                   all          3         19      0.917      0.932      0.995      0.886
            light_blue          1          5       0.77          1      0.995      0.955
             dark_blue          1          7      0.981          1      0.995      0.858
                others          1          7          1      0.797      0.995      0.846
Speed: 0.3ms preprocess, 49.3ms inference, 

✅ Data FP16 berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/openvino_fp16_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,38.81073
inference/openvino_fp16_fps,25.76607
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.995
metrics/mAP50-95(B),0.88647


🔒 WandB Session Closed.


38.81073474884033

### Yolo11n 256

In [23]:
# 1. Coba Size 256 (Sweet Spot?)
run_experiment(
    model_name='yolo11n.pt',  # Pastikan ini menunjuk ke checkpoint terbaikmu
    exp_name='yolo11n_256',
    img_size=256,             # <--- Turunkan resolusi
    use_safe_aug=True
)


🚀 START EXPERIMENT: yolo11n_256 | Size: 256 | Safe Aug: True


🛡️ Mengaktifkan SAFE MODE...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/bottle_cap/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolo11n_256, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15,

lr/pg0,▁▂▂▃▃▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▆▅▅▄▄▃▃▂
lr/pg1,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▇▆▆▆▅▄▄▃▃▂
lr/pg2,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇███████████▇▇▇▆▆▆▅▅▄▄▄▃▂
metrics/mAP50(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▄▄▅▅▅▅▅▆▆▆▇▇▇████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▃▃▄▄▄▅▅▆▆▆▇▇████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇██████████
metrics/recall(B),▂▂▂▅▅▅▅▅▆▆███████████████████▂▂▁▁▁▂▂▂▃▃▃
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


📊 Running Validation...
Ultralytics 8.3.231  Python-3.11.9 torch-2.4.1+cpu CPU (12th Gen Intel Core i5-1235U)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 1.60.4 ms, read: 2.30.6 MB/s, size: 43.4 KB)
val: Scanning G:\My Drive\We Work The Talk\Ada Mata - Machine Learning Engineer\ada_mata_mle\datasets\bottle_cap\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 3/3 2.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 4.5it/s 0.2s
                   all          3         19      0.956      0.488      0.995      0.817
            light_blue          1          5      0.867          1      0.995      0.792
             dark_blue          1          7          1      0.271      0.995      0.827
                others          1          7          1      0.193      0.995      0.832
Speed: 0.3ms preprocess, 51.6ms inference, 

✅ Data benchmark berhasil dikirim ke WandB!


inference/latency_ms,▁
inference/onnx_fps,▁
model/size_mb,▁
val/mAP50,▁
inference/latency_ms,27.11026
inference/onnx_fps,36.88641
lr/pg0,4e-05
lr/pg1,4e-05
lr/pg2,4e-05
metrics/mAP50(B),0.98917
metrics/mAP50-95(B),0.81524


🔒 Sesi WandB ditutup.


27.110257148742676